In [ ]:
import pandas as pd
import psycopg2 as ps
import dotenv
import os

In [ ]:
dotenv.load_dotenv(".conf")

In [ ]:
from datetime import datetime
from uuid import uuid4
str(datetime.now()), uuid4().hex

In [ ]:
C = ps.connect(
    host=os.getenv("DB_HOST").strip(),
    port=os.getenv("DB_PORT").strip(),
    user=os.getenv("DB_USER").strip(),
    password=os.getenv("DB_PASSWORD").strip(),
    database=os.getenv("DB_NAME").strip()
)

There are two options to do this: total count mode, or do the count for each day, and then take the average over all dates.

Total count mode

Fixed query with potential missing buckets when there is no data

In [ ]:
pd.read_sql_query("""
    with bucketed as (
        select 
          date_bin(
              '15 minutes', 
              record_time, 
              date_trunc('day', record_time) --'2022-08-11'::date
          ) as trunc
        from keyevents
    ), times as (
        select 
          extract('hour' from trunc) as hour,
          extract('minute' from trunc) as minute
        from bucketed
    ), res as (
        select
            hour,
            minute,
            count(*)
        from times group by hour, minute
        order by hour, minute asc
    )
    select * from res
""", con=C)

In [ ]:
pd.read_sql_query("""
    with fixed_buckets as (
        select 
          time_of_day
        from generate_series(
          '2022-08-01'::date,
          '2022-08-02'::date,
          '15 minutes'::interval
        ) time_of_day
    ), fixed_times as (
        select 
            distinct
          extract('hour' from time_of_day)::smallint as hour,
          extract('minute' from time_of_day)::smallint as minute
        from fixed_buckets
    ), bucketed as (
        select 
          date_bin(
              '15 minutes', 
              record_time, 
              date_trunc('day', record_time) --'2022-08-11'::date
          ) as trunc
        from Keystrokes
    ), strokes_times as (
        select 
          extract('hour' from trunc)::smallint as hour,
          extract('minute' from trunc)::smallint as minute
        from bucketed
    ), final_form as (
        select
            hour,
            minute,
            (
                select 
                    count(*) as strokes_count
                from strokes_times T
                where T.hour=REF.hour
                and T.minute=REF.minute
            )
        from fixed_times REF
        order by hour, minute asc
    ) select * from final_form
""", con=C)

In [ ]:
pd.read_sql_query("""
    with timeline as (
        select 
          min(date_trunc('day', record_time)) as first_date,
          max(date_trunc('day', record_time)) last_date
          from Keystrokes
    ) select * from timeline
""", con=C)

In [ ]:
pd.read_sql_query("""
    with timeline as (
        select 
          min(date_trunc('day', record_time)) as first_date,
          max(date_trunc('day', record_time)) last_date
          from Keystrokes
    ),
    fixed_buckets as (
        select 
          time_of_day
        from timeline T, generate_series(
          T.first_date::date,
          T.last_date::date,
          '15 minutes'::interval
        ) time_of_day
    ), fixed_times as (
        select  distinct
            date_trunc('day', time_of_day)::date as day,
          extract('hour' from time_of_day)::smallint as hour,
          extract('minute' from time_of_day)::smallint as minute
        from fixed_buckets
    ), bucketed as (
        select 
          date_bin(
              '15 minutes', 
              record_time, 
              date_trunc('day', record_time) --'2022-08-11'::date
          ) as trunc,
          date_trunc('day', record_time) as full_date
        from key_events
    ), strokes_times as (
        select 
          extract('hour' from trunc)::smallint as hour,
          extract('minute' from trunc)::smallint as minute,
          full_date
        from bucketed
    ), final_form as (
        select
            day,
            hour,
            minute,
            (
                select 
                    count(*) as strokes_count
                from strokes_times T
                where T.hour=REF.hour
                and T.minute=REF.minute
                and T.full_date::date=REF.day::date
            )
        from fixed_times REF
        order by day, hour, minute asc
    ), averaged as (
        select 
            hour, minute, avg(strokes_count) as strokes_count
        from final_form
        group by hour, minute
    ) select 
        hour, minute, --lpad(hour::text, 2, '0')|| ':' || lpad(minute::text, 2, '0'), 
        strokes_count 
    from averaged 
""", con=C)

In [ ]:
pd.read_sql_query("""
    with 
    bucketed as (
        select 
          date_bin(
              '15 minutes', 
              record_time, 
              date_trunc('day', record_time) 
          ) as trunc,
          date_trunc('day', record_time) as day
        from keyevents
    ), strokes_times as (
        select 
          extract('hour' from trunc)::smallint as hour,
          extract('minute' from trunc)::smallint as minute,
          day
        from bucketed
    ), final_form as (
        select
            day,
            hour,
            minute,
            count(*) as strokes_count
        from strokes_times REF
        group by day, hour, minute
        order by day, hour, minute asc
    ), averaged as (
        select 
            hour, minute, avg(strokes_count) as strokes_count
        from final_form
        group by hour, minute
    ) select 
        hour, minute, --lpad(hour::text, 2, '0')|| ':' || lpad(minute::text, 2, '0'), 
        strokes_count 
    from averaged 
    order by hour, minute
""", con=C)

If moving forward with DBT, a good test could be to compute the sum and compare with the total sum

In [ ]:
pd.read_sql_query("""
    with daily_buckets as (
        select 
          timeline.* 
        from generate_series(
          B.ts - '${seconds * params.availableXBars} seconds'::interval,
          B.ts,
          '${seconds} seconds'::interval
        ) timeline
    )
""", con=C)